# Prediction for Control

### Import Packages

In [1]:
using ForwardDiff
using DelimitedFiles
using LinearAlgebra
using MathOptInterface
using JuMP
using SparseArrays
using Plots
using OSQP
using PyCall
push!(PyVector(pyimport("sys")."path"), "../scripts")
python_utils = pyimport("python_utils")
np = pyimport("numpy");

### Include Julia Scripts

In [45]:
include("../scripts/path_handling.jl")
include("../scripts/utils.jl")
include("../scripts/costs.jl")
include("../scripts/dynamics.jl")
include("../scripts/interface.jl")
include("../scripts/optimization_constraints.jl")
include("../scripts/mpc.jl")
include("../scripts/visualization.jl");

### Load Data

In [46]:
# load data
env = python_utils.load_data_set("/home/zxc/codes/MATS/experiments/processed/nuScenes_val_full.pkl");

### Load Model and Settings

In [47]:
# load model
model_path = "../../experiments/nuScenes/models/models_21_Jul_2020_10_25_10_full_zeroRrows_batch8_double_fixed_a_norm"
mats, hyperparams = load_model(model_path, env, ts=11)

# model prediction settings
num_modes = 1
pred_settings = PredictionSettings(mats, hyperparams, env, num_modes);

### Scene Selection

In [48]:
# select scene
scene_num = 24 # corresponds to 23 in python indexing
scene = Scene(env, scene_num);
println(scene.robot.x[3:10])

[865.598060978798, 869.2313796833996, 872.8644701896831, 876.6201509561033, 880.4207558250864, 884.373652835415, 888.5498075752839, 892.8720610426615]


### Initialize and Construct MPC Problem

In [49]:
if !isdir("data")
    mkdir("data")
end

# select time interval
first_ts = 3 
last_ts = scene.timesteps;
# last_ts = 4

# get path data
x_coefs_var, y_coefs_var, breaks_var = load_splines()
path_obj = SplinePath(x_coefs_var, y_coefs_var, breaks_var)

# robot initial state
q0 = [scene.robot.x[first_ts], scene.robot.y[first_ts], scene.robot.θ[first_ts], scene.robot.v[first_ts], 0]
println("x", scene.robot.x[first_ts])
println("y", scene.robot.y[first_ts])
q0[5] = find_best_s(q0, path_obj, enable_global_search=true)

# println("q0:", q0)

# MPC parameters, constraints and settings
control_limits_obj = ControlLimits(.7, -.7, 4., -5., 12., 0.)
dynamics_obj = DynamicsModel(4, 2, control_limits_obj)
vals_obj = MPCValues(path_obj, n_modes=num_modes, N=25, k_c=4, q0=q0, n_obs=length(scene.nodes))

# make first predictions for obstacle constraints
init_node_obstacles!(scene, vals_obj)
Aps, Bps, gps, q_pred0, nodes_present = predicted_dynamics(pred_settings, scene_num, first_ts)
u_pred = get_recorded_robot_controls(pred_settings, scene_num, first_ts)
q_pred = [predict_future_states(q_pred0, u_pred, Aps, Bps, gps, j) for j = 1:vals_obj.n_modes]

update_obstacles_from_predictions!(q_pred, nodes_present, vals_obj, scene)

# initial solution guess
qs, us = initial_guess(vals_obj);
# println("qs:", qs)
# println("us:", size(us))

x865.598060978798
y1423.7636933758395
Local search too narrow. Performing global search.
Global search successfully found a better projection.


In [50]:
# construct problem
model, params, variables, mpc_values = construct_problem(dynamics_obj, vals_obj, scene, qs, us, verbose=false);
# println(mpc_values.q0)

### Solve MPC

In [51]:
x_traj = Array{Float64,1}()
y_traj = Array{Float64,1}()
θ_traj = Array{Float64,1}()
v_traj = Array{Float64,1}()
s_traj = Array{Float64,1}()

ω_traj = []
a_traj = []
vs_traj = []

x_pred = []
y_pred = []
θ_pred = []
v_pred = []
s_pred = []

ω_pred = []
a_pred = []
vs_pred = []

agent_states = []

ss =[0:0.01:138;]
spline_idces = find_spline_interval.(ss, (path_obj,))
x_path = spline_x.(ss, (path_obj,), spline_idces)
y_path = spline_y.(ss, (path_obj,), spline_idces)

n_iter_val = 3 # number of solve/relinearization iterations per timestep
    
qs = []
us = []

problem_infeasible = false

if !isdir("figs")
    mkdir("figs")
end

for ts = first_ts:last_ts
    println("--------------------------------------------")
    println("Starting iteration ", ts - first_ts + 1, "!")
    

    if ts == first_ts
        n_iter = 15
    else
        n_iter = n_iter_val
    end

    for j = 1:n_iter
        println("q0:,", mpc_values.q0)
        println("u0:,", mpc_values.u0)
        optimize!(model)
        
        println("Termination Status: ", termination_status(model))
        println("Relinearization loop: ", j)

        output_qs = [value(q) for q in variables.q]
        output_us = [value(u) for u in variables.u]
        
#         println("output_qs:", output_qs[:, 1:3])
        # println("output_us:", output_us[:, 1:3])
        # println("us:", size(output_us))


        if (sum(isnan.(output_qs)) > 0) || (sum(isnan.(output_us)) > 0)
            println("Infeasible. Skipping to next iteration.")
            problem_infeasible = true
            break
        else
            qs = output_qs
            us = output_us
        end    

        relinearize!(mpc_values, qs, us)

        flush(stdout)
    end

    if problem_infeasible
        println("Coasting along previous solution.")
        flush(stdout)
        problem_infeasible = false
        qs, us = update_problem!(mpc_values, qs, us)

        push!(x_traj, qs[1,1])
        push!(y_traj, qs[2,1])
        push!(θ_traj, qs[3,1])
        push!(v_traj, qs[4,1])
        push!(s_traj, qs[5,1])

        push!(ω_traj, us[1,1])
        push!(a_traj, us[2,1])
        push!(vs_traj, us[3,1])

        push!(x_pred, qs[1,:])
        push!(y_pred, qs[2,:])
        push!(s_pred, qs[5,:])

        plot!(x_path, y_path, label="path", color=:blue, legend=:topright)
        scatter!([x_traj[ts]], [y_traj[ts]], marker=:dot, color=:black, ms=3, label="pos")
        scatter!(x_pred[ts], y_pred[ts], marker=:dot, color=:red, ms=1, label="pred", dpi=300)

        continue
    end

    push!(x_traj, qs[1,1])
    push!(y_traj, qs[2,1])
    push!(θ_traj, qs[3,1])
    push!(v_traj, qs[4,1])
    push!(s_traj, qs[5,1])

    push!(ω_traj, us[1,1])
    push!(a_traj, us[2,1])
    push!(vs_traj, us[3,1])

    push!(x_pred, qs[1,:])
    push!(y_pred, qs[2,:])
    push!(θ_pred, qs[3,:])
    push!(v_pred, qs[4,:])
    push!(s_pred, qs[5,:])
    
    push!(ω_pred, us[1,:])
    push!(a_pred, qs[2,:])
    push!(vs_pred, qs[3,:])

    # visualization
    plot()
    viz_path!(x_path, y_path)
    viz_solution!(qs, vals_obj)   
    viz_obstacle!(vals_obj)
    fig_settings!((x_path[1] - 100, x_path[end] + 100), (y_path[2] - 125, y_path[2] + 25))
    savefig("figs/" * string(ts - first_ts + 1) * ".png")
    
    # save data
    save_obstacle_data(ts - first_ts + 1, vals_obj)
    if (ts - first_ts + 1) == 11
        np.save("data/x_traj", x_traj)
        np.save("data/y_traj", y_traj)
        np.save("data/θ_traj", θ_traj)
        np.save("data/v_traj", v_traj)
        np.save("data/s_traj", s_traj)
        
        np.save("data/ω_traj", ω_traj)
        np.save("data/a_traj", a_traj)
        np.save("data/vs_traj", vs_traj)
        
        np.save("data/x_pred", x_pred)
        np.save("data/y_pred", y_pred)
        np.save("data/θ_pred", θ_pred)
        np.save("data/v_pred", v_pred)
        np.save("data/s_pred", s_pred)
        
        np.save("data/ω_pred", ω_pred)
        np.save("data/a_pred", a_pred)
        np.save("data/vs_pred", vs_pred)
    end
    
    
    pred_horizon = hyperparams.get("prediction_horizon")
    pred_settings.env.scenes[scene_num].robot.data.data[ts+1:ts+pred_horizon+1, 1] = qs[1, 1:pred_horizon+1]  # x 
    pred_settings.env.scenes[scene_num].robot.data.data[ts+1:ts+pred_horizon+1, 2] = qs[2, 1:pred_horizon+1]  # y
    pred_settings.env.scenes[scene_num].robot.data.data[ts+1:ts+pred_horizon+1, 9] = qs[3, 1:pred_horizon+1]  # θ
    pred_settings.env.scenes[scene_num].robot.data.data[ts+1:ts+pred_horizon, 10] = us[1, 1:pred_horizon]     # ω
    pred_settings.env.scenes[scene_num].robot.data.data[ts+1:ts+pred_horizon+1, 11] = qs[4, 1:pred_horizon+1] # v
    pred_settings.env.scenes[scene_num].robot.data.data[ts+1:ts+pred_horizon, 12] = us[2, 1:pred_horizon]     # a
    
    Aps, Bps, gps, q_pred0, nodes_present = predicted_dynamics(pred_settings, scene_num, ts+1)
    
    u_pred = us[1:2, 1:pred_horizon]' # use control from optimization solution
    q_pred = [predict_future_states(q_pred0, u_pred, Aps, Bps, gps, j) for j = 1:vals_obj.n_modes]
    
    update_obstacles_from_predictions!(q_pred, nodes_present, vals_obj, scene, iter=ts-first_ts+2)
    println("qs:", qs[:, 1:3])
    update_problem!(mpc_values, qs, us)
    model, params, variables, mpc_values = construct_problem(dynamics_obj, vals_obj, scene, qs, us, verbose=false);
end

--------------------------------------------
Starting iteration 1!
q0:,[865.598060978798, 1423.7636933758395, -0.6786788288239216, 9.30225337459255, 41.45]
u0:,[0.0, 0.0, 0.0]
Termination Status: OPTIMAL
Relinearization loop: 1
q0:,[865.598060978798, 1423.7636933758395, -0.6786788288239216, 9.30225337459255, 41.45]
u0:,[0.0, 0.0, 0.0]
Termination Status: OPTIMAL
Relinearization loop: 2
q0:,[865.598060978798, 1423.7636933758395, -0.6786788288239216, 9.30225337459255, 41.45]
u0:,[0.0, 0.0, 0.0]
Termination Status: OPTIMAL
Relinearization loop: 3
q0:,[865.598060978798, 1423.7636933758395, -0.6786788288239216, 9.30225337459255, 41.45]
u0:,[0.0, 0.0, 0.0]
Termination Status: OPTIMAL
Relinearization loop: 4
q0:,[865.598060978798, 1423.7636933758395, -0.6786788288239216, 9.30225337459255, 41.45]
u0:,[0.0, 0.0, 0.0]
Termination Status: OPTIMAL
Relinearization loop: 5
q0:,[865.598060978798, 1423.7636933758395, -0.6786788288239216, 9.30225337459255, 41.45]
u0:,[0.0, 0.0, 0.0]
Termination Status:

q0:,[878.9630047706873, 1415.1023445235917, -0.4522557756980212, 9.615439170265894, 57.44120668529031]
u0:,[-0.29406628937300006, -4.962054493689704, 10.135171776734882]
Termination Status: OPTIMAL
Relinearization loop: 3
qs:[878.9630059881783 881.1831656434671 883.4329766188685; 1415.1023452461502 1414.0046525068155 1412.5667533609615; -0.45223906668743846 -0.4519818439772041 -0.6265432551630327; 9.615438738283663 10.165323084576892 11.165230241160055; 57.44120668416558 59.80900311363906 62.65600236450006]
--------------------------------------------
Starting iteration 8!
q0:,[881.1831656434671, 1414.0046525068155, -0.4519818439772041, 10.165323084576892, 59.80900311363906]
u0:,[0.0009667827685179996, 2.1995413569711038, 10.071185722389165]
Termination Status: OPTIMAL
Relinearization loop: 1
q0:,[881.1831656434671, 1414.0046525068155, -0.4519818439772041, 10.165323084576892, 59.80900311363906]
u0:,[0.0009667827685179996, 2.1995413569711038, 10.071185722389165]
Termination Status: OPTI

q0:,[896.3787822769704, 1403.0599927214394, -0.6565956415835262, 10.454317742049781, 78.88798938326266]
u0:,[-0.0016908216942261764, -0.4989685146045778, 10.198215860675356]
Termination Status: OPTIMAL
Relinearization loop: 3
qs:[896.3787822763454 898.4442575968158 900.4994767221168; 1403.0599927215303 1401.479123305991 1400.1314163838188; -0.6565956417312252 -0.655101438900309 -0.5159698279145697; 10.45431774066062 10.328896730134657 9.294329701087907; 78.88798938351526 81.39980714550627 83.85166479398256]
--------------------------------------------
Starting iteration 16!
q0:,[898.4442575968158, 1401.479123305991, -0.655101438900309, 10.328896730134657, 81.39980714550627]
u0:,[0.005976807383450244, -0.5016840383278733, 9.847271046953752]
Termination Status: OPTIMAL
Relinearization loop: 1
q0:,[898.4442575968158, 1401.479123305991, -0.655101438900309, 10.328896730134657, 81.39980714550627]
u0:,[0.005976807383450244, -0.5016840383278733, 9.847271046953752]
Termination Status: OPTIMAL
R

MethodError: MethodError: no method matching spline_x(::Float64, ::SplinePath, ::Nothing)
Closest candidates are:
  spline_x(::Any, ::SplinePath, !Matched::Integer) at /home/zxc/codes/MATS/MPC/scripts/path_handling.jl:67